# <div style="text-align: right;font-family:Times New Roman">Приложение Б-4</div>
# <div style="text-align: center;font-family:Times New Roman">Обучение моделей</div>

# Model training
# Обучение моделей
---
In this notebook convolutional neural networks for tree species, forest types and bonitet modeling are trained, tested and used for vegetation mapping

В данном блокноте происходит обучение и тестирование сверточных нейронных сетей для моделирования преобладающей породы, типов леса и бонитета, а также при помощи применения этих моделей создаются карты растительности

### Importing dependencies
### Импорт необходимых библиотек

In [1]:
import os
import random
import re
import numpy as np
from glob import glob
import pandas as pd
import pickle
from osgeo import ogr, osr

import fiona
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterio.merge
import rasterio.fill
from rasterio.io import MemoryFile
from rasterio.enums import Resampling
from rasterio.windows import Window
from rasterio.transform import Affine

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras import optimizers

C:\ProgramData\Anaconda3\envs\geo\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### Data preparation
### Подготовка данных

In [7]:
#generating tiles / генерация тайлов
tiles = []
x1 = 0
y1 = 0
x2 = 128
y2 = 128
for i in range (0,632,1):
    for i in range(0,1089,1):
        tiles.append((x1,y1,x2,y2))
        y1 +=128
        y2 +=128
    y1=0
    y2=128
    x1+=128
    x2+=128

In [3]:
tiles

[(0, 0, 128, 128),
 (0, 128, 128, 256),
 (0, 256, 128, 384),
 (0, 384, 128, 512),
 (0, 512, 128, 640),
 (0, 640, 128, 768),
 (0, 768, 128, 896),
 (0, 896, 128, 1024),
 (0, 1024, 128, 1152),
 (0, 1152, 128, 1280),
 (0, 1280, 128, 1408),
 (0, 1408, 128, 1536),
 (0, 1536, 128, 1664),
 (0, 1664, 128, 1792),
 (0, 1792, 128, 1920),
 (0, 1920, 128, 2048),
 (0, 2048, 128, 2176),
 (0, 2176, 128, 2304),
 (0, 2304, 128, 2432),
 (0, 2432, 128, 2560),
 (0, 2560, 128, 2688),
 (0, 2688, 128, 2816),
 (0, 2816, 128, 2944),
 (0, 2944, 128, 3072),
 (0, 3072, 128, 3200),
 (0, 3200, 128, 3328),
 (0, 3328, 128, 3456),
 (0, 3456, 128, 3584),
 (0, 3584, 128, 3712),
 (0, 3712, 128, 3840),
 (0, 3840, 128, 3968),
 (0, 3968, 128, 4096),
 (0, 4096, 128, 4224),
 (0, 4224, 128, 4352),
 (0, 4352, 128, 4480),
 (0, 4480, 128, 4608),
 (0, 4608, 128, 4736),
 (0, 4736, 128, 4864),
 (0, 4864, 128, 4992),
 (0, 4992, 128, 5120),
 (0, 5120, 128, 5248),
 (0, 5248, 128, 5376),
 (0, 5376, 128, 5504),
 (0, 5504, 128, 5632),
 (0, 

In [10]:
#shuffling samples / перемешивание тайлов
samples = list(range(len(tiles)))

In [11]:
np.random.shuffle(samples)
samples

[39835,
 426550,
 181911,
 523918,
 488131,
 623641,
 486702,
 119430,
 399948,
 102815,
 287068,
 149306,
 213356,
 439911,
 467365,
 500018,
 127822,
 517009,
 309426,
 53235,
 132422,
 398907,
 241276,
 556842,
 445047,
 92312,
 555997,
 197758,
 213660,
 635660,
 326046,
 472495,
 141066,
 419357,
 398797,
 562182,
 433605,
 38414,
 508973,
 419171,
 14484,
 384330,
 542195,
 538670,
 70437,
 539147,
 375822,
 658917,
 382712,
 57240,
 85812,
 9050,
 60179,
 230579,
 391133,
 677658,
 117911,
 114601,
 157164,
 118836,
 56161,
 590190,
 128775,
 84756,
 571050,
 340052,
 299968,
 576363,
 621205,
 352035,
 172891,
 452411,
 395769,
 610073,
 383245,
 546980,
 551739,
 115822,
 479245,
 285077,
 376648,
 669662,
 76927,
 512250,
 334901,
 33281,
 58962,
 204392,
 651473,
 272610,
 336997,
 573293,
 314586,
 247632,
 97359,
 52936,
 680685,
 241736,
 354126,
 414547,
 264241,
 211309,
 516658,
 93563,
 579361,
 31297,
 638802,
 518128,
 563793,
 48554,
 369623,
 241225,
 7199,
 16922

In [6]:
len(samples)

688248

In [13]:
with open(r"E:\Work\CorrProizv\samples.pickle", "wb") as fp:
    pickle.dump(samples, fp)

In [8]:
with open(r"E:\Work\CorrProizv\samples.pickle", "rb") as fp:
    samples = pickle.load(fp)

In [9]:
#splitting data to train, validation and test / разделение данных на тренировочный, валидационный и тестовый набор
train = samples[:400000]
val = samples[400000:500000]
test = samples[500000:]

##### x data / данные-предикторы

In [2]:
#getting all data sources / получение всех источников данных
path = r'F:\Work\CorrProizv\final_noadj\\'
files = glob(path+'*.tif')
files.append(r'F:\Work\CorrProizv\DEM.tif')
files.append(r'F:\Work\CorrProizv\Landcover.tif')
files.append(r'F:\Work\CorrProizv\Hansen_mosaic.tif')
files

['F:\\Work\\CorrProizv\\final_noadj\\B1.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B11.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B12.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B2.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B3.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B4.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B5.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B6.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B7.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B8.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B8A.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B9.tif',
 'F:\\Work\\CorrProizv\\DEM.tif',
 'F:\\Work\\CorrProizv\\Landcover.tif',
 'F:\\Work\\CorrProizv\\Hansen_mosaic.tif']

In [3]:
#creating numpy array / создание массива numpy
x_train_conv = np.empty((80896, 139392, 15), dtype = 'uint16')

In [6]:
#reading all x files / чтение всех данных-предикторов
for i in range(len(files)):
    with rio.open(files[i]) as bnd:
        container = x_train_conv[:,:,i:i+1]
        container[:] = np.transpose(np.pad(bnd.read(), ((0,0),(0,89),(0,62)), mode='constant', constant_values=0), axes=[1, 2, 0]).astype('uint16')
        print(files[i])

F:\Work\CorrProizv\final_noadj\B1.tif
F:\Work\CorrProizv\final_noadj\B11.tif
F:\Work\CorrProizv\final_noadj\B12.tif
F:\Work\CorrProizv\final_noadj\B2.tif
F:\Work\CorrProizv\final_noadj\B3.tif
F:\Work\CorrProizv\final_noadj\B4.tif
F:\Work\CorrProizv\final_noadj\B5.tif
F:\Work\CorrProizv\final_noadj\B6.tif
F:\Work\CorrProizv\final_noadj\B7.tif
F:\Work\CorrProizv\final_noadj\B8.tif
F:\Work\CorrProizv\final_noadj\B8A.tif
F:\Work\CorrProizv\final_noadj\B9.tif
F:\Work\CorrProizv\DEM.tif
F:\Work\CorrProizv\Landcover.tif
F:\Work\CorrProizv\Hansen_mosaic.tif


In [3]:
x_train_conv.shape

(80896, 139392, 15)

In [24]:
np.save(r'F:\Work\CorrProizv\x_train_conv.npy', x_train_conv)

In [2]:
x_train_conv = np.load(r'F:\Work\CorrProizv\x_train_conv.npy')

In [ ]:
#generating x train tiles / генерация тайлов данных-предикторов тренировочного набора
x_train = np.empty((len(train), 128, 128, 15), dtype = 'uint16')

In [10]:
for i in range(len(train)):
    c = tiles[train[i]]
    container = x_train[i:i+1,:,:,:]
    container[:] = x_train_conv[c[0]:c[2], c[1]:c[3],:]

In [33]:
x_train.shape

(400000, 128, 128, 15)

In [34]:
np.save(r'F:\Work\CorrProizv\x_train.npy', x_train)

In [13]:
#generating x validation tiles / генерация тайлов данных-предикторов валидационного набора
x_val = np.empty((len(val), 128, 128, 15), dtype = 'uint16')

In [15]:
for i in range(len(val)):
    c = tiles[val[i]]
    container = x_val[i:i+1,:,:,:]
    container[:] = x_train_conv[c[0]:c[2], c[1]:c[3],:]

In [14]:
x_val.shape

(100000, 128, 128, 15)

In [16]:
np.save(r'F:\Work\CorrProizv\x_val.npy', x_val)

In [24]:
#generating x test tiles / генерация тайлов данных-предикторов тестового набора
x_test = np.empty((len(test), 128, 128, 15), dtype = 'uint16')

In [25]:
for i in range(len(test)):
    c = tiles[test[i]]
    container = x_test[i:i+1,:,:,:]
    container[:] = x_train_conv[c[0]:c[2], c[1]:c[3],:]

In [26]:
x_test.shape

(188248, 128, 128, 15)

In [27]:
np.save(r'F:\Work\CorrProizv\x_test.npy', x_test)

##### y data for tree species model / обучающие данные для модели преобладающих пород

In [2]:
y = r'F:\Work\CorrProizv\y_spec2.tif'

In [3]:
#reading y file / чтение обучающих данных
with rio.open(y) as bnd:
    y_train_conv = bnd.read().astype('uint16')
    final_trans = bnd.transform

In [4]:
y_train_conv.shape

(1, 80807, 139330)

In [5]:
y_train_conv = np.pad(y_train_conv, ((0,0),(0,89),(0,62)), mode='constant', constant_values=0)

In [6]:
y_train_conv = np.transpose(y_train_conv, axes=[1, 2, 0])

In [7]:
y_train_conv.shape

(80896, 139392, 1)

In [13]:
#generating y train tiles / генерация тайлов обучающих данных тренировочного набора
y_train = np.empty((len(train), 128, 128, 1), dtype = 'uint16')

In [14]:
for i in range(len(train)):
    c = tiles[train[i]]
    container = y_train[i:i+1,:,:,:]
    container[:] = y_train_conv[c[0]:c[2], c[1]:c[3],:]

In [15]:
y_train.shape

(400000, 128, 128, 1)

In [16]:
y_train = utils.to_categorical(y_train, num_classes=14, dtype = 'float16')

In [18]:
y_train.shape

(400000, 128, 128, 14)

In [19]:
np.save(r'F:\Work\CorrProizv\y_train.npy', y_train)

In [20]:
#generating y validation tiles / генерация тайлов обучающих данных валидационного набора
y_val = np.empty((len(val), 128, 128, 1), dtype = 'uint16')

In [21]:
for i in range(len(val)):
    c = tiles[val[i]]
    container = y_val[i:i+1,:,:,:]
    container[:] = y_train_conv[c[0]:c[2], c[1]:c[3],:]

In [22]:
y_val.shape

(100000, 128, 128, 1)

In [23]:
y_val = utils.to_categorical(y_val, num_classes=14, dtype = 'float16')

In [24]:
y_val.shape

(100000, 128, 128, 14)

In [28]:
np.save(r'E:\Work\CorrProizv\y_val.npy', y_val)

In [29]:
#generating y test tiles / генерация тайлов обучающих данных тестового набора
y_test = np.empty((len(test), 128, 128, 1), dtype = 'uint16')

In [30]:
for i in range(len(test)):
    c = tiles[test[i]]
    container = y_test[i:i+1,:,:,:]
    container[:] = y_train_conv[c[0]:c[2], c[1]:c[3],:]

In [31]:
y_test.shape

(188248, 128, 128, 1)

In [32]:
y_test = utils.to_categorical(y_test, num_classes=14, dtype = 'float16')

In [33]:
y_test.shape

(188248, 128, 128, 14)

In [34]:
np.save(r'F:\Work\CorrProizv\y_test.npy', y_test)

##### y data for forest types model / обучающие данные для модели типов леса

In [5]:
#reading y file / чтение обучающих данных
y = r'F:\Work\CorrProizv\y_type.tif'

In [6]:
with rio.open(y) as bnd:
    y_train_conv = bnd.read()
    final_trans = bnd.transform

In [7]:
y_train_conv = np.pad(y_train_conv, ((0,0),(0,89),(0,62)), mode='constant', constant_values=0)

In [8]:
y_train_conv = np.transpose(y_train_conv, axes=[1, 2, 0])

In [9]:
y_train_conv.shape

(80896, 139392, 1)

In [12]:
#generating y train tiles / генерация тайлов обучающих данных тренировочного набора
y_train = np.empty((len(train), 128, 128, 1), dtype = 'uint16')

In [13]:
for i in range(len(train)):
    c = tiles[train[i]]
    container = y_train[i:i+1,:,:,:]
    container[:] = y_train_conv[c[0]:c[2], c[1]:c[3],:]

In [14]:
y_train = utils.to_categorical(y_train, num_classes=10, dtype = 'float16')

In [15]:
np.save(r'F:\Work\CorrProizv\y_types_train.npy', y_train)

In [16]:
#generating y validation tiles / генерация тайлов обучающих данных валидационного набора
y_val = np.empty((len(val), 128, 128, 1), dtype = 'uint16')

In [17]:
for i in range(len(val)):
    c = tiles[val[i]]
    container = y_val[i:i+1,:,:,:]
    container[:] = y_train_conv[c[0]:c[2], c[1]:c[3],:]

In [18]:
y_val = utils.to_categorical(y_val, num_classes=10, dtype = 'float16')

In [19]:
np.save(r'F:\Work\CorrProizv\y_types_val.npy', y_val)

In [20]:
#generating y test tiles / генерация тайлов обучающих данных тестового набора
y_test = np.empty((len(test), 128, 128, 1), dtype = 'uint16')

In [21]:
for i in range(len(test)):
    c = tiles[test[i]]
    container = y_test[i:i+1,:,:,:]
    container[:] = y_train_conv[c[0]:c[2], c[1]:c[3],:]

In [22]:
y_test = utils.to_categorical(y_test, num_classes=10, dtype = 'float16')

In [23]:
np.save(r'F:\Work\CorrProizv\y_types_test.npy', y_test)

##### y data for forest bonitet model / обучающие данные для модели бонитетов леса

In [2]:
#reading y file / чтение обучающих данных
y = r'F:\Work\CorrProizv\y_bon.tif'

In [3]:
with rio.open(y) as bnd:
    y_train_conv = bnd.read()
    final_trans = bnd.transform

In [4]:
y_train_conv = np.pad(y_train_conv, ((0,0),(0,89),(0,62)), mode='constant', constant_values=0)

In [5]:
y_train_conv = np.transpose(y_train_conv, axes=[1, 2, 0])

In [6]:
y_train_conv.shape

(80896, 139392, 1)

In [10]:
#generating y train tiles / генерация тайлов обучающих данных тренировочного набора
y_train = np.empty((len(train), 128, 128, 1), dtype = 'uint16')

In [11]:
for i in range(len(train)):
    c = tiles[train[i]]
    container = y_train[i:i+1,:,:,:]
    container[:] = y_train_conv[c[0]:c[2], c[1]:c[3],:]

In [12]:
y_train = utils.to_categorical(y_train, num_classes=8, dtype = 'float16')

In [13]:
np.save(r'F:\Work\CorrProizv\y_bon_train.npy', y_train)

In [14]:
#generating y validation tiles / генерация тайлов обучающих данных валидационного набора
y_val = np.empty((len(val), 128, 128, 1), dtype = 'uint16')

In [15]:
for i in range(len(val)):
    c = tiles[val[i]]
    container = y_val[i:i+1,:,:,:]
    container[:] = y_train_conv[c[0]:c[2], c[1]:c[3],:]

In [16]:
y_val = utils.to_categorical(y_val, num_classes=8, dtype = 'float16')

In [17]:
np.save(r'F:\Work\CorrProizv\y_bon_val.npy', y_val)

In [18]:
#generating y test tiles / генерация тайлов обучающих данных тестового набора
y_test = np.empty((len(test), 128, 128, 1), dtype = 'uint16')

In [19]:
for i in range(len(test)):
    c = tiles[test[i]]
    container = y_test[i:i+1,:,:,:]
    container[:] = y_train_conv[c[0]:c[2], c[1]:c[3],:]

In [20]:
y_test = utils.to_categorical(y_test, num_classes=8, dtype = 'float16')

In [21]:
np.save(r'F:\Work\CorrProizv\y_bon_test.npy', y_test)

### Tree species modelling 
### Моделирование преобладающих пород
##### U-NET

In [2]:
#loading train and validation data / загрузка тренировочных и валидационных данных
x_train = np.load(r'F:\Work\CorrProizv\x_train.npy')

In [3]:
y_train = np.load(r'F:\Work\CorrProizv\y_train.npy')

In [4]:
x_val = np.load(r'F:\Work\CorrProizv\x_val.npy')

In [5]:
y_val = np.load(r'F:\Work\CorrProizv\y_val.npy')

In [36]:
# variation 1, without batch normalization, performed bad / вариант 1 без пакетной нормализации, показал плохой результат
from tensorflow.keras import layers, models, optimizers
inputs = layers.Input(shape = (128, 128, 15))

conv1 = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
conv1 = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = layers.Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = layers.Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = layers.Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = layers.Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = layers.Dropout(0.5)(conv4)
pool4 = layers.MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = layers.Conv2D(1024, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = layers.Conv2D(1024, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = layers.Dropout(0.5)(conv5)

up6 = layers.Conv2D(512, (2, 2), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(drop5))
merge6 = layers.concatenate([drop4,up6], axis = 3)
conv6 = layers.Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = layers.Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = layers.Conv2D(256, (2, 2), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv6))
merge7 = layers.concatenate([conv3,up7], axis = 3)
conv7 = layers.Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = layers.Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = layers.Conv2D(128, (2, 2), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv7))
merge8 = layers.concatenate([conv2,up8], axis = 3)
conv8 = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = layers.Conv2D(64, (2, 2), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv8))
merge9 = layers.concatenate([conv1,up9], axis = 3)
conv9 = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
#conv9 = layers.Conv2D(2, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
#conv10 = layers.Conv2D(1, 1, activation = 'softmax', padding = 'same')(conv9)
conv10 = layers.Conv2D(14, (1, 1), activation = 'softmax', padding = 'same')(conv9)

model = models.Model(inputs = inputs, outputs = conv10)

model.compile(optimizer = optimizers.Adam(learning_rate=0.01), 
              loss = 'categorical_crossentropy', 
              metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall'),
                       tf.keras.metrics.MeanAbsoluteError(name = 'MAE'),
                       tf.keras.metrics.AUC(name='auc')])
"""keras.optimizers.Adam(learning_rate = 0.01)"""

'keras.optimizers.Adam(learning_rate = 0.01)'

In [66]:
#added batch normalization, transpose / добавлена пакетная нормализация

from tensorflow.keras import layers, models, optimizers
inputs = layers.Input(shape = (128, 128, 15))

conv1 = layers.Conv2D(64, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(inputs)
norm1 = layers.BatchNormalization()(conv1)
relu1 = layers.Activation('relu')(norm1)
conv1 = layers.Conv2D(64, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(relu1)
norm1 = layers.BatchNormalization()(conv1)
relu1 = layers.Activation('relu')(norm1)
pool1 = layers.MaxPooling2D(pool_size=(2, 2))(relu1)
drop1 = layers.Dropout(0.1)(pool1)
conv2 = layers.Conv2D(128, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(drop1)
norm2 = layers.BatchNormalization()(conv2)
relu2 = layers.Activation('relu')(norm2)
conv2 = layers.Conv2D(128, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(relu2)
norm2 = layers.BatchNormalization()(conv2)
relu2 = layers.Activation('relu')(norm2)
pool2 = layers.MaxPooling2D(pool_size=(2, 2))(relu2)
drop2 = layers.Dropout(0.1)(pool2)
conv3 = layers.Conv2D(256, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(drop2)
norm3 = layers.BatchNormalization()(conv3)
relu3 = layers.Activation('relu')(norm3)
conv3 = layers.Conv2D(256, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(relu3)
norm3 = layers.BatchNormalization()(conv3)
relu3 = layers.Activation('relu')(norm3)
pool3 = layers.MaxPooling2D(pool_size=(2, 2))(relu3)
drop3 = layers.Dropout(0.1)(pool3)
conv4 = layers.Conv2D(512, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(drop3)
norm4 = layers.BatchNormalization()(conv4)
relu4 = layers.Activation('relu')(norm4)
conv4 = layers.Conv2D(512, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(relu4)
norm4 = layers.BatchNormalization()(conv4)
relu4 = layers.Activation('relu')(norm4)
pool4 = layers.MaxPooling2D(pool_size=(2, 2))(relu4)
drop4 = layers.Dropout(0.1)(pool4)

conv5 = layers.Conv2D(1024, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(drop4)
norm5 = layers.BatchNormalization()(conv5)
relu5 = layers.Activation('relu')(norm5)
conv5 = layers.Conv2D(1024, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(relu5)
norm5 = layers.BatchNormalization()(conv5)
relu5 = layers.Activation('relu')(norm5)

up6 = layers.Conv2DTranspose(512, (2, 2), strides = (2, 2), padding = 'same', kernel_initializer = 'he_normal')(relu5)
merge6 = layers.concatenate([relu4,up6], axis = 3)
drop6 = layers.Dropout(0.1)(merge6)
conv6 = layers.Conv2D(512, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(drop6)
norm6 = layers.BatchNormalization()(conv6)
relu6 = layers.Activation('relu')(norm6)
conv6 = layers.Conv2D(512, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(relu6)
norm6 = layers.BatchNormalization()(conv6)
relu6 = layers.Activation('relu')(norm6)

up7 = layers.Conv2DTranspose(256, (2, 2), strides = (2, 2), padding = 'same', kernel_initializer = 'he_normal')(relu6)
merge7 = layers.concatenate([relu3,up7], axis = 3)
drop7 = layers.Dropout(0.1)(merge7)
conv7 = layers.Conv2D(256, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(drop7)
norm7 = layers.BatchNormalization()(conv7)
relu7 = layers.Activation('relu')(norm7)
conv7 = layers.Conv2D(256, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(relu7)
norm7 = layers.BatchNormalization()(conv7)
relu7 = layers.Activation('relu')(norm7)

up8 = layers.Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same', kernel_initializer = 'he_normal')(relu7)
merge8 = layers.concatenate([relu2,up8], axis = 3)
drop8 = layers.Dropout(0.1)(merge8)
conv8 = layers.Conv2D(128, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(drop8)
norm8 = layers.BatchNormalization()(conv8)
relu8 = layers.Activation('relu')(norm8)
conv8 = layers.Conv2D(128, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(relu8)
norm8 = layers.BatchNormalization()(conv8)
relu8 = layers.Activation('relu')(norm8)

up9 = layers.Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same', kernel_initializer = 'he_normal')(relu8)
merge9 = layers.concatenate([relu1,up9], axis = 3)
drop9 = layers.Dropout(0.1)(merge9)
conv9 = layers.Conv2D(64, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(drop9)
norm9 = layers.BatchNormalization()(conv9)
relu9 = layers.Activation('relu')(norm9)
conv9 = layers.Conv2D(64, (3, 3), padding = 'same', kernel_initializer = 'he_normal')(relu9)
norm9 = layers.BatchNormalization()(conv9)
relu9 = layers.Activation('relu')(norm9)

#conv9 = layers.Conv2D(2, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
#conv10 = layers.Conv2D(1, 1, activation = 'softmax', padding = 'same')(conv9)
conv10 = layers.Conv2D(14, (1, 1), activation = 'softmax', padding = 'same')(relu9)

model = models.Model(inputs = inputs, outputs = conv10)

model.compile(optimizer = optimizers.Adam(learning_rate=0.01), 
              loss = 'categorical_crossentropy', 
              metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall'),
                       tf.keras.metrics.MeanAbsoluteError(name = 'MAE'),
                       tf.keras.metrics.AUC(name='auc')])

In [3]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 128, 128, 15 0                                            
__________________________________________________________________________________________________
conv2d_161 (Conv2D)             (None, 128, 128, 64) 8704        input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 64) 256         conv2d_161[0][0]                 
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 64) 0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [10]:
checkpoint_filepath = r'E:\Work\CorrProizv\unet_species4_1.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001)

In [11]:
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, X_ds, y_ds, batch_size, *args, **kwargs):
        self.batch_size = batch_size
        self.X_ds = X_ds
        self.y_ds = y_ds

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.X_ds) / self.batch_size))

    def __getitem__(self, index):
        # returns one batch
        X = self.X_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        y = self.y_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        for i in range(len(X)):
            X[i] = X[i] * np.broadcast_to((1-y[i][:,:,0:1]), X[i].shape)
        return X, y

batch_size = 16
training_generator = CustomDataGen(X_ds = x_train, y_ds = y_train, batch_size = batch_size)
val_generator = CustomDataGen(X_ds = x_val, y_ds = y_val, batch_size = batch_size)

In [12]:
#model training / обучение модели
history = model.fit(training_generator, epochs = 15, validation_data = val_generator, callbacks=[model_checkpoint_callback, early_stopping, reduce_lr])

Epoch 1/15
25000/25000 [==============================] - 39833s 2s/step - loss: 0.3403 - accuracy: 0.8746 - precision: 0.9334 - recall: 0.8317 - MAE: 0.0239 - auc: 0.9949 - val_loss: 0.3491 - val_accuracy: 0.8691 - val_precision: 0.9423 - val_recall: 0.8140 - val_MAE: 0.0251 - val_auc: 0.9948
Epoch 2/15
25000/25000 [==============================] - 40975s 2s/step - loss: 0.3381 - accuracy: 0.8752 - precision: 0.9332 - recall: 0.8330 - MAE: 0.0237 - auc: 0.9949 - val_loss: 0.3302 - val_accuracy: 0.8783 - val_precision: 0.9427 - val_recall: 0.8274 - val_MAE: 0.0242 - val_auc: 0.9954
Epoch 3/15
25000/25000 [==============================] - 40737s 2s/step - loss: 0.3360 - accuracy: 0.8758 - precision: 0.9331 - recall: 0.8340 - MAE: 0.0236 - auc: 0.9950 - val_loss: 0.3351 - val_accuracy: 0.8769 - val_precision: 0.9309 - val_recall: 0.8390 - val_MAE: 0.0231 - val_auc: 0.9947
Epoch 4/15
25000/25000 [==============================] - 39910s 2s/step - loss: 0.3316 - accuracy: 0.8772 - precis

In [13]:
history.history

{'loss': [0.34030860662460327,
  0.33812659978866577,
  0.3359527587890625,
  0.3315733075141907,
  0.3302082419395447,
  0.32915371656417847,
  0.32809051871299744,
  0.3268054723739624,
  0.3250522315502167,
  0.32316771149635315,
  0.32151734828948975,
  0.3200722336769104,
  0.3188088834285736,
  0.31764858961105347,
  0.3164924383163452],
 'accuracy': [0.8745589852333069,
  0.8751804828643799,
  0.8758410215377808,
  0.8772223591804504,
  0.8776218891143799,
  0.8779910802841187,
  0.8783040642738342,
  0.8787323236465454,
  0.8792669773101807,
  0.8797903060913086,
  0.8802225589752197,
  0.8806324005126953,
  0.8809046149253845,
  0.8813313841819763,
  0.8816524744033813],
 'precision': [0.9334188103675842,
  0.9331514239311218,
  0.9331044554710388,
  0.9331106543540955,
  0.9331945776939392,
  0.9330999851226807,
  0.9330139756202698,
  0.9330273866653442,
  0.932870090007782,
  0.9328112006187439,
  0.9329028725624084,
  0.9329884052276611,
  0.9331595301628113,
  0.933132886

##### DEEPLAB

In [6]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [23]:
def DeeplabV3Plus(image_size, num_classes):
    model_input = tf.keras.Input(shape=(image_size, image_size, 15))
    resnet50 = tf.keras.applications.ResNet50(
        weights=None, include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), activation = 'softmax',  padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=128, num_classes=14)
model.compile(optimizer = optimizers.Adam(learning_rate=0.01), 
              loss = 'categorical_crossentropy', 
              metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall'),
                       tf.keras.metrics.MeanAbsoluteError(name = 'MAE'),
                       tf.keras.metrics.AUC(name='auc')])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 128, 128, 15 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 15) 0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   47104       conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [40]:
checkpoint_filepath = r'E:\Work\CorrProizv\deeplab_species1_1.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001)

In [25]:
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, X_ds, y_ds, batch_size, *args, **kwargs):
        self.batch_size = batch_size
        self.X_ds = X_ds
        self.y_ds = y_ds

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.X_ds) / self.batch_size))

    def __getitem__(self, index):
        # returns one batch
        X = self.X_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        y = self.y_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        for i in range(len(X)):
            X[i] = X[i] * np.broadcast_to((1-y[i][:,:,0:1]), X[i].shape)
        return X, y

batch_size = 16
training_generator = CustomDataGen(X_ds = x_train, y_ds = y_train, batch_size = batch_size)
val_generator = CustomDataGen(X_ds = x_val, y_ds = y_val, batch_size = batch_size)

In [ ]:
#model showed better performance than U-Net / модель показала лучшие результаты, чем U-Net
#only last 5 epochs are showed / показаны только результаты последних 5 эпох
history = model.fit(training_generator, epochs = 15, validation_data = val_generator, callbacks=[model_checkpoint_callback, early_stopping, reduce_lr])

Epoch 1/15
25000/25000 [==============================] - 26444s 1s/step - loss: 0.2995 - accuracy: 0.8873 - precision: 0.9324 - recall: 0.8531 - MAE: 0.0217 - auc: 0.9960 - val_loss: 0.3074 - val_accuracy: 0.8848 - val_precision: 0.9315 - val_recall: 0.8504 - val_MAE: 0.0219 - val_auc: 0.9957
Epoch 2/15
25000/25000 [==============================] - 25508s 1s/step - loss: 0.2983 - accuracy: 0.8878 - precision: 0.9324 - recall: 0.8538 - MAE: 0.0216 - auc: 0.9960 - val_loss: 0.3148 - val_accuracy: 0.8813 - val_precision: 0.9345 - val_recall: 0.8419 - val_MAE: 0.0226 - val_auc: 0.9956
Epoch 3/15
25000/25000 [==============================] - 26451s 1s/step - loss: 0.2972 - accuracy: 0.8882 - precision: 0.9324 - recall: 0.8545 - MAE: 0.0215 - auc: 0.9960 - val_loss: 0.3133 - val_accuracy: 0.8826 - val_precision: 0.9302 - val_recall: 0.8476 - val_MAE: 0.0226 - val_auc: 0.9956
Epoch 4/15
25000/25000 [==============================] - 26035s 1s/step - loss: 0.2960 - accuracy: 0.8887 - precis

In [30]:
history

### Testing tree species model
### Тестирование модели преобладающих пород

In [2]:
#loading test data / загрузка тестовых данных
x_test = np.load(r'F:\Work\CorrProizv\x_test.npy')

In [3]:
y_test = np.load(r'F:\Work\CorrProizv\y_test.npy')

In [7]:
#loading model / загрузка модели
model = tf.keras.models.load_model(r'E:\Work\CorrProizv\deeplab_species1.hdf5')

In [8]:
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, X_ds, y_ds, batch_size, *args, **kwargs):
        self.batch_size = batch_size
        self.X_ds = X_ds
        self.y_ds = y_ds

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.X_ds) / self.batch_size))

    def __getitem__(self, index):
        # returns one batch
        X = self.X_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        y = self.y_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        for i in range(len(X)):
            X[i] = X[i] * np.broadcast_to((1-y[i][:,:,0:1]), X[i].shape)
        return X, y

batch_size = 16
test_generator = CustomDataGen(X_ds = x_test, y_ds = y_test, batch_size = batch_size)

In [9]:
#testing / тестирование
results = model.evaluate(test_generator)

11765/11765 [==============================] - 3204s 272ms/step - loss: 0.3119 - accuracy: 0.8830 - precision: 0.9354 - recall: 0.8435 - MAE: 0.0228 - auc: 0.9957


### Tree species map
### Карта преобладающих пород

In [10]:
#loading model / загрузка модели
model = tf.keras.models.load_model(r'E:\Work\CorrProizv\deeplab_species1_1.hdf5')

In [12]:
#generating tiles / генерация тайлов
tiles = []
x1 = 0
y1 = 0
x2 = 128
y2 = 128
for i in range (0,632,1):
    for i in range(0,1089,1):
        tiles.append((x1,y1,x2,y2))
        y1 +=128
        y2 +=128
    y1=0
    y2=128
    x1+=128
    x2+=128

In [11]:
#loading train, validation and test split / загрузка тренировочного, валидационного и тестового наборов
with open(r"E:\Work\CorrProizv\samples.pickle", "rb") as fp:
    samples = pickle.load(fp)

In [13]:
train = samples[:400000]
val = samples[400000:500000]
test = samples[500000:]

In [14]:
#loading y data raster as a reference / загрузка файла с обучающими данными для привязки итогового растра
y = r'F:\Work\CorrProizv\y_spec2.tif'

In [15]:
yband = rio.open(y)

In [16]:
yband.shape

(80807, 139330)

In [17]:
yband.dtypes

('int16',)

In [18]:
#creating prediction array / создание массива для моделирования
y_pred = np.zeros((80896, 139392), dtype = 'int16')

In [19]:
#modelling tree species for train data / моделирование преобладающей породы для тренировочных даннных
x_train = np.load(r'F:\Work\CorrProizv\x_train.npy')

In [20]:
for i in range(len(train)):
    prediction = model.predict(x_train[i:i+1])
    prediction = np.squeeze(np.argmax(prediction, axis=-1))
    t = train[i]
    t = tiles[t]
    y_pred[t[0]:t[2],t[1]:t[3]] = prediction

In [21]:
x_train = None

In [22]:
#modelling tree species for validation data / моделирование преобладающей породы для валидационных даннных
x_val = np.load(r'F:\Work\CorrProizv\x_val.npy')

In [23]:
for i in range(len(val)):
    prediction = model.predict(x_val[i:i+1])
    prediction = np.squeeze(np.argmax(prediction, axis=-1))
    t = val[i]
    t = tiles[t]
    y_pred[t[0]:t[2],t[1]:t[3]] = prediction

In [24]:
#modelling tree species for test data / моделирование преобладающей породы для тестовых даннных
x_test = np.load(r'F:\Work\CorrProizv\x_test.npy')

In [25]:
for i in range(len(test)):
    prediction = model.predict(x_test[i:i+1])
    prediction = np.squeeze(np.argmax(prediction, axis=-1))
    t = test[i]
    t = tiles[t]
    y_pred[t[0]:t[2],t[1]:t[3]] = prediction

In [26]:
y_pred.shape

(80896, 139392)

In [27]:
y_pred = y_pred[:80807, :139330]

In [28]:
#saving map into raster file / сохранение карты в растровый файл
with rio.open(
    r'F:\Work\CorrProizv\y_pred_deeplab.tif',
    'w',
    driver='GTiff',
    height=y_pred.shape[0],
    width=y_pred.shape[1],
    count=1,
    dtype=y_pred.dtype,
    compress = 'deflate',
    PREDICTOR = 1,
    ZLEVEL=9,
    crs=yband.crs,
    transform=yband.transform,
    nodata = 0
) as outfile:
    outfile.write(y_pred, 1)

### Forest types modelling 
### Моделирование типов леса

In [25]:
#loading train and validation data / загрузка тренировочных и валидационных данных
x_train = np.load(r'F:\Work\CorrProizv\x_train.npy')

In [4]:
y_train = np.load(r'F:\Work\CorrProizv\y_types_train.npy')

In [26]:
x_val = np.load(r'F:\Work\CorrProizv\x_val.npy')

In [6]:
y_val = np.load(r'F:\Work\CorrProizv\y_types_val.npy')

In [27]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=tf.keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [29]:
def DeeplabV3Plus(image_size, num_classes):
    model_input = tf.keras.Input(shape=(image_size, image_size, 15))
    resnet50 = tf.keras.applications.ResNet50(
        weights=None, include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), activation = 'softmax',  padding="same")(x)
    return tf.keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=128, num_classes=10)
model.compile(optimizer = optimizers.Adam(learning_rate=0.01), 
              loss = 'categorical_crossentropy', 
              metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall'),
                       tf.keras.metrics.MeanAbsoluteError(name = 'MAE'),
                       tf.keras.metrics.AUC(name='auc')])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 15 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 15) 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   47104       conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [30]:
checkpoint_filepath = r'E:\Work\CorrProizv\deeplab_types.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001)

In [31]:
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, X_ds, y_ds, batch_size, *args, **kwargs):
        self.batch_size = batch_size
        self.X_ds = X_ds
        self.y_ds = y_ds

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.X_ds) / self.batch_size))

    def __getitem__(self, index):
        # returns one batch
        X = self.X_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        y = self.y_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        for i in range(len(X)):
            X[i] = X[i] * np.broadcast_to((1-y[i][:,:,0:1]), X[i].shape)
        return X, y

batch_size = 16
training_generator = CustomDataGen(X_ds = x_train, y_ds = y_train, batch_size = batch_size)
val_generator = CustomDataGen(X_ds = x_val, y_ds = y_val, batch_size = batch_size)

In [32]:
history = model.fit(training_generator, epochs = 10, validation_data = val_generator, callbacks=[model_checkpoint_callback, early_stopping, reduce_lr])

Epoch 1/10
25000/25000 [==============================] - 28255s 1s/step - loss: 0.3951 - accuracy: 0.8478 - precision: 0.9781 - recall: 0.7859 - MAE: 0.0360 - auc: 0.9902 - val_loss: 0.3741 - val_accuracy: 0.8546 - val_precision: 0.9842 - val_recall: 0.7889 - val_MAE: 0.0350 - val_auc: 0.9914


C:\ProgramData\Anaconda3\envs\geo\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/10
25000/25000 [==============================] - 21470s 859ms/step - loss: 0.3672 - accuracy: 0.8584 - precision: 0.9737 - recall: 0.7968 - MAE: 0.0341 - auc: 0.9917 - val_loss: 0.3630 - val_accuracy: 0.8600 - val_precision: 0.9655 - val_recall: 0.8060 - val_MAE: 0.0327 - val_auc: 0.9915
Epoch 3/10
25000/25000 [==============================] - 27364s 1s/step - loss: 0.3578 - accuracy: 0.8622 - precision: 0.9710 - recall: 0.8017 - MAE: 0.0335 - auc: 0.9922 - val_loss: 0.5043 - val_accuracy: 0.8229 - val_precision: 0.9107 - val_recall: 0.7891 - val_MAE: 0.0381 - val_auc: 0.9861
Epoch 4/10
25000/25000 [==============================] - 30472s 1s/step - loss: 0.3424 - accuracy: 0.8681 - precision: 0.9689 - recall: 0.8087 - MAE: 0.0325 - auc: 0.9929 - val_loss: 0.3484 - val_accuracy: 0.8659 - val_precision: 0.9518 - val_recall: 0.8190 - val_MAE: 0.0317 - val_auc: 0.9926
Epoch 5/10
25000/25000 [==============================] - 23521s 941ms/step - loss: 0.3385 - accuracy: 0.8696 - 

In [34]:
history.history

{'loss': [0.39514750242233276,
  0.367192804813385,
  0.35779261589050293,
  0.34243571758270264,
  0.3385457694530487,
  0.3356682062149048,
  0.33329635858535767,
  0.32882159948349,
  0.3271735608577728,
  0.3258102834224701],
 'accuracy': [0.8478060960769653,
  0.8584378361701965,
  0.8622170090675354,
  0.8680524826049805,
  0.8695633411407471,
  0.8708953261375427,
  0.871849775314331,
  0.8735653758049011,
  0.8742461800575256,
  0.8747755289077759],
 'precision': [0.9781430959701538,
  0.973706841468811,
  0.9709579348564148,
  0.9688681960105896,
  0.9676061868667603,
  0.9667657017707825,
  0.9659939408302307,
  0.9648558497428894,
  0.9642929434776306,
  0.9637888669967651],
 'recall': [0.7859051823616028,
  0.7968304753303528,
  0.8016785979270935,
  0.8087245225906372,
  0.8110524415969849,
  0.8127419948577881,
  0.8139996528625488,
  0.8166502118110657,
  0.8176752328872681,
  0.8184917569160461],
 'MAE': [0.0360274575650692,
  0.034141622483730316,
  0.03346353024244308

### Testing forest types model
### Тестирование модели типов леса

In [2]:
#loading test data / загрузка тестовых данных
x_test = np.load(r'F:\Work\CorrProizv\x_test.npy')

In [7]:
y_test = np.load(r'F:\Work\CorrProizv\y_types_test.npy')

In [8]:
#loading model / загрузка модели
model = tf.keras.models.load_model(r'E:\Work\CorrProizv\deeplab_types.hdf5')

In [9]:
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, X_ds, y_ds, batch_size, *args, **kwargs):
        self.batch_size = batch_size
        self.X_ds = X_ds
        self.y_ds = y_ds

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.X_ds) / self.batch_size))

    def __getitem__(self, index):
        # returns one batch
        X = self.X_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        y = self.y_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        for i in range(len(X)):
            X[i] = X[i] * np.broadcast_to((1-y[i][:,:,0:1]), X[i].shape)
        return X, y

batch_size = 16
test_generator = CustomDataGen(X_ds = x_test, y_ds = y_test, batch_size = batch_size)

In [10]:
#testing / тестирование
results = model.evaluate(test_generator)

11765/11765 [==============================] - 3286s 279ms/step - loss: 0.3264 - accuracy: 0.8751 - precision: 0.9651 - recall: 0.8182 - MAE: 0.0312 - auc: 0.9936


### Forest types map
### Карта типов леса

In [2]:
#loading model / загрузка модели
model = tf.keras.models.load_model(r'E:\Work\CorrProizv\deeplab_types.hdf5')

In [3]:
#generating tiles / генерация тайлов
tiles = []
x1 = 0
y1 = 0
x2 = 128
y2 = 128
for i in range (0,632,1):
    for i in range(0,1089,1):
        tiles.append((x1,y1,x2,y2))
        y1 +=128
        y2 +=128
    y1=0
    y2=128
    x1+=128
    x2+=128

In [4]:
#loading train, validation and test split / загрузка тренировочного, валидационного и тестового наборов
with open(r"E:\Work\CorrProizv\samples.pickle", "rb") as fp:
    samples = pickle.load(fp)

In [5]:
train = samples[:400000]
val = samples[400000:500000]
test = samples[500000:]

In [6]:
#loading y data raster as a reference / загрузка файла с обучающими данными для привязки итогового растра
y = r'F:\Work\CorrProizv\y_spec2.tif'
yband = rio.open(y)

In [7]:
#creating prediction array / создание массива для моделирования
y_pred = np.zeros((80896, 139392), dtype = 'int16')

In [8]:
#modelling tree species for train data / моделирование преобладающей породы для тренировочных даннных
x_train = np.load(r'F:\Work\CorrProizv\x_train.npy')

In [9]:
for i in range(len(train)):
    prediction = model.predict(x_train[i:i+1])
    prediction = np.squeeze(np.argmax(prediction, axis=-1))
    t = train[i]
    t = tiles[t]
    y_pred[t[0]:t[2],t[1]:t[3]] = prediction

In [10]:
#modelling tree species for validation data / моделирование преобладающей породы для валидационных даннных
x_val = np.load(r'F:\Work\CorrProizv\x_val.npy')

In [11]:
for i in range(len(val)):
    prediction = model.predict(x_val[i:i+1])
    prediction = np.squeeze(np.argmax(prediction, axis=-1))
    t = val[i]
    t = tiles[t]
    y_pred[t[0]:t[2],t[1]:t[3]] = prediction

In [12]:
#modelling tree species for test data / моделирование преобладающей породы для тестовых даннных
x_test = np.load(r'F:\Work\CorrProizv\x_test.npy')

In [13]:
for i in range(len(test)):
    prediction = model.predict(x_test[i:i+1])
    prediction = np.squeeze(np.argmax(prediction, axis=-1))
    t = test[i]
    t = tiles[t]
    y_pred[t[0]:t[2],t[1]:t[3]] = prediction

In [14]:
y_pred = y_pred[:80807, :139330]

In [15]:
#saving map into raster file / сохранение карты в растровый файл
with rio.open(
    r'F:\Work\CorrProizv\y_pred_types.tif',
    'w',
    driver='GTiff',
    height=y_pred.shape[0],
    width=y_pred.shape[1],
    count=1,
    dtype=y_pred.dtype,
    compress = 'deflate',
    PREDICTOR = 1,
    ZLEVEL=9,
    crs=yband.crs,
    transform=yband.transform,
    nodata = 0
) as outfile:
    outfile.write(y_pred, 1)

### Forest bonitet modelling 
### Моделирование бонитета леса

In [23]:
#loading train and validation data / загрузка тренировочных и валидационных данных
x_train = np.load(r'F:\Work\CorrProizv\x_train.npy')

In [ ]:
y_train = np.load(r'F:\Work\CorrProizv\y_bon_train.npy')

In [24]:
x_val = np.load(r'F:\Work\CorrProizv\x_val.npy')

In [ ]:
y_val = np.load(r'F:\Work\CorrProizv\y_bon_val.npy')

In [25]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=tf.keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [27]:
def DeeplabV3Plus(image_size, num_classes):
    model_input = tf.keras.Input(shape=(image_size, image_size, 15))
    resnet50 = tf.keras.applications.ResNet50(
        weights=None, include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), activation = 'softmax',  padding="same")(x)
    return tf.keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=128, num_classes=8)
model.compile(optimizer = optimizers.Adam(learning_rate=0.01), 
              loss = 'categorical_crossentropy', 
              metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall'),
                       tf.keras.metrics.MeanAbsoluteError(name = 'MAE'),
                       tf.keras.metrics.AUC(name='auc')])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 15 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 15) 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   47104       conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [28]:
checkpoint_filepath = r'E:\Work\CorrProizv\deeplab_bon.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001)

In [29]:
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, X_ds, y_ds, batch_size, *args, **kwargs):
        self.batch_size = batch_size
        self.X_ds = X_ds
        self.y_ds = y_ds

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.X_ds) / self.batch_size))

    def __getitem__(self, index):
        # returns one batch
        X = self.X_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        y = self.y_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        for i in range(len(X)):
            X[i] = X[i] * np.broadcast_to((1-y[i][:,:,0:1]), X[i].shape)
        return X, y

batch_size = 16
training_generator = CustomDataGen(X_ds = x_train, y_ds = y_train, batch_size = batch_size)
val_generator = CustomDataGen(X_ds = x_val, y_ds = y_val, batch_size = batch_size)

In [30]:
history = model.fit(training_generator, epochs = 10, validation_data = val_generator, callbacks=[model_checkpoint_callback, early_stopping, reduce_lr])

Epoch 1/10
25000/25000 [==============================] - 17713s 708ms/step - loss: 0.3195 - accuracy: 0.8648 - precision: 0.9548 - recall: 0.7973 - MAE: 0.0414 - auc: 0.9921 - val_loss: 0.3501 - val_accuracy: 0.8606 - val_precision: 0.8866 - val_recall: 0.8478 - val_MAE: 0.0385 - val_auc: 0.9904


C:\ProgramData\Anaconda3\envs\geo\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/10
25000/25000 [==============================] - 17984s 719ms/step - loss: 0.2943 - accuracy: 0.8736 - precision: 0.9488 - recall: 0.8130 - MAE: 0.0392 - auc: 0.9932 - val_loss: 0.3581 - val_accuracy: 0.8517 - val_precision: 0.9516 - val_recall: 0.7942 - val_MAE: 0.0419 - val_auc: 0.9897
Epoch 3/10
25000/25000 [==============================] - 17051s 682ms/step - loss: 0.2794 - accuracy: 0.8797 - precision: 0.9450 - recall: 0.8238 - MAE: 0.0378 - auc: 0.9939 - val_loss: 0.2738 - val_accuracy: 0.8811 - val_precision: 0.9286 - val_recall: 0.8430 - val_MAE: 0.0364 - val_auc: 0.9941
Epoch 4/10
25000/25000 [==============================] - 16652s 666ms/step - loss: 0.2758 - accuracy: 0.8811 - precision: 0.9428 - recall: 0.8280 - MAE: 0.0374 - auc: 0.9940 - val_loss: 0.2716 - val_accuracy: 0.8820 - val_precision: 0.9364 - val_recall: 0.8349 - val_MAE: 0.0372 - val_auc: 0.9942
Epoch 5/10
25000/25000 [==============================] - 16909s 676ms/step - loss: 0.2733 - accuracy: 0.8

In [33]:
history.history

{'loss': [0.3194510340690613,
  0.2943100333213806,
  0.2793913185596466,
  0.27578482031822205,
  0.2732796370983124,
  0.2711236774921417,
  0.2672930955886841,
  0.2658865749835968,
  0.2648017704486847,
  0.26372748613357544],
 'accuracy': [0.8647693991661072,
  0.8735628724098206,
  0.8796509504318237,
  0.8811419606208801,
  0.8821361064910889,
  0.8830742835998535,
  0.8846959471702576,
  0.8852378726005554,
  0.8856346607208252,
  0.8862057328224182],
 'precision': [0.9548406004905701,
  0.9487602710723877,
  0.9449564218521118,
  0.9427663683891296,
  0.9412365555763245,
  0.9397451281547546,
  0.9375885128974915,
  0.9370201230049133,
  0.9364311099052429,
  0.9357175230979919],
 'recall': [0.7973067760467529,
  0.8130154013633728,
  0.8238339424133301,
  0.8280373215675354,
  0.8309144377708435,
  0.8336891531944275,
  0.838403582572937,
  0.8398922085762024,
  0.8411339521408081,
  0.8427765369415283],
 'MAE': [0.04141265153884888,
  0.039200033992528915,
  0.03781045973300

### Testing forest bonitet model
### Тестирование модели бонитета леса

In [2]:
#loading test data / загрузка тестовых данных
x_test = np.load(r'F:\Work\CorrProizv\x_test.npy')

In [3]:
y_test = np.load(r'F:\Work\CorrProizv\y_bon_test.npy')

In [4]:
#loading model / загрузка модели
model = tf.keras.models.load_model(r'E:\Work\CorrProizv\deeplab_bon.hdf5')

In [5]:
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, X_ds, y_ds, batch_size, *args, **kwargs):
        self.batch_size = batch_size
        self.X_ds = X_ds
        self.y_ds = y_ds

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.X_ds) / self.batch_size))

    def __getitem__(self, index):
        # returns one batch
        X = self.X_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        y = self.y_ds[index*self.batch_size:(index+1)*self.batch_size].astype('float32')
        for i in range(len(X)):
            X[i] = X[i] * np.broadcast_to((1-y[i][:,:,0:1]), X[i].shape)
        return X, y

batch_size = 16
test_generator = CustomDataGen(X_ds = x_test, y_ds = y_test, batch_size = batch_size)

In [6]:
#testing / тестирование
results = model.evaluate(test_generator)

11765/11765 [==============================] - 2677s 227ms/step - loss: 0.2658 - accuracy: 0.8849 - precision: 0.9349 - recall: 0.8421 - MAE: 0.0363 - auc: 0.9944


### Map of forest bonitets
### Карта бонитетов леса

In [2]:
#loading model / загрузка модели
model = tf.keras.models.load_model(r'E:\Work\CorrProizv\deeplab_bon.hdf5')

In [3]:
#generating tiles / генерация тайлов
tiles = []
x1 = 0
y1 = 0
x2 = 128
y2 = 128
for i in range (0,632,1):
    for i in range(0,1089,1):
        tiles.append((x1,y1,x2,y2))
        y1 +=128
        y2 +=128
    y1=0
    y2=128
    x1+=128
    x2+=128

In [4]:
#loading train, validation and test split / загрузка тренировочного, валидационного и тестового наборов
with open(r"E:\Work\CorrProizv\samples.pickle", "rb") as fp:
    samples = pickle.load(fp)

In [5]:
train = samples[:400000]
val = samples[400000:500000]
test = samples[500000:]

In [6]:
#loading y data raster as a reference / загрузка файла с обучающими данными для привязки итогового растра
y = r'F:\Work\CorrProizv\y_spec2.tif'
yband = rio.open(y)

In [7]:
#creating prediction array / создание массива для моделирования
y_pred = np.zeros((80896, 139392), dtype = 'int16')

In [8]:
#modelling tree species for train data / моделирование преобладающей породы для тренировочных даннных
x_train = np.load(r'F:\Work\CorrProizv\x_train.npy')

In [9]:
for i in range(len(train)):
    prediction = model.predict(x_train[i:i+1])
    prediction = np.squeeze(np.argmax(prediction, axis=-1))
    t = train[i]
    t = tiles[t]
    y_pred[t[0]:t[2],t[1]:t[3]] = prediction

In [10]:
#modelling tree species for validation data / моделирование преобладающей породы для валидационных даннных
x_val = np.load(r'F:\Work\CorrProizv\x_val.npy')

In [11]:
for i in range(len(val)):
    prediction = model.predict(x_val[i:i+1])
    prediction = np.squeeze(np.argmax(prediction, axis=-1))
    t = val[i]
    t = tiles[t]
    y_pred[t[0]:t[2],t[1]:t[3]] = prediction

In [12]:
#modelling tree species for test data / моделирование преобладающей породы для тестовых даннных
x_test = np.load(r'F:\Work\CorrProizv\x_test.npy')

In [13]:
for i in range(len(test)):
    prediction = model.predict(x_test[i:i+1])
    prediction = np.squeeze(np.argmax(prediction, axis=-1))
    t = test[i]
    t = tiles[t]
    y_pred[t[0]:t[2],t[1]:t[3]] = prediction

In [14]:
y_pred = y_pred[:80807, :139330]

In [15]:
#saving map into raster file / сохранение карты в растровый файл
with rio.open(
    r'F:\Work\CorrProizv\y_pred_bon.tif',
    'w',
    driver='GTiff',
    height=y_pred.shape[0],
    width=y_pred.shape[1],
    count=1,
    dtype=y_pred.dtype,
    compress = 'deflate',
    PREDICTOR = 1,
    ZLEVEL=9,
    crs=yband.crs,
    transform=yband.transform,
    nodata = 0
) as outfile:
    outfile.write(y_pred, 1)

#### Tools for tile generation, semantic segmentation and mapping of the modeling results from this notebook are now avaliable in a Python library named [Remote Sensing Processor](https://github.com/simonreise/remote-sensing-processor)
#### Инструменты для генерации тайлов, семантической сегментации и картирования результатов моделирования, использованные в этом блокноте доступны в библиотеке Python [Remote Sensing Processor](https://github.com/simonreise/remote-sensing-processor)